In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/was-your-stay-worth-its-price/sample_submission.csv
/kaggle/input/was-your-stay-worth-its-price/reviews.csv
/kaggle/input/was-your-stay-worth-its-price/train.csv
/kaggle/input/was-your-stay-worth-its-price/test.csv


## Installing tensorflow Decision tree forest library
We need to explicitly install these libraries since it's not supported on kaggle yet

In [4]:

pip install tensorflow_decision_forests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 48.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard


In [5]:
#pip install tensorflow-addons

Note: you may need to restart the kernel to use updated packages.


In [112]:
import tensorflow_decision_forests as tfdf
import tensorflow as tf
#import tensorflow_addons as tfa
try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

from IPython.core.magic import register_line_magic
from IPython.display import Javascript

In [167]:
df_train = pd.read_csv('/kaggle/input/was-your-stay-worth-its-price/train.csv')
df_train.head()

,id,host_1,host_2,host_3,host_4,host_5,host_6,host_7,host_8,property_1,...,property_15,property_16,property_max_booking,property_min_booking,property_price,reviews_each_month,reviews_last_30_days,reviews_total,total_listings,max_rating_class
0,d23eea91dda3cfc8,5,t,t,f,within an hour,99%,100%,111,3.0,...,t,38,1125,2,$792.00,1.43,0,36,116.0,location
1,91aa7023cbfe98f8,6,t,t,f,within a day,80%,0%,8,9.0,...,f,36,1124,31,$166.00,0.27,0,29,7.0,cleanliness
2,ad155140fab30327,6,t,t,t,within an hour,100%,100%,2,6.0,...,t,45,1125,1,$178.00,4.09,2,24,1.0,checkin
3,f90f94bf3b78a43e,6,t,t,f,within a day,80%,28%,1,6.0,...,f,26,1125,3,$61.00,0.29,1,5,1.0,accuracy
4,cb32a50ba2199d4c,1,t,t,f,within an hour,100%,100%,1,0.0,...,f,73,8,2,$340.00,1.32,0,8,0.0,rating


In [168]:
# df_train.info()

### Checking how many missing values are there 

In [169]:
# df_train.isnull().sum().sort_values(ascending=False)

## Pre-processing 

Tf Decision Trees will work with both categorical and numerical data and account for any missing data as well. 

In [170]:
# Encode the categorical labels as integers.
#
# Details:
# This stage is necessary if your classification label is represented as a
# string since Keras expects integer classification labels.
# When using `pd_dataframe_to_tf_dataset` (see below), this step can be skipped.

# Name of the label column.
label = "max_rating_class"

classes = df_train[label].unique().tolist()
print(f"Label classes: {classes}")


df_train[label] = df_train[label].map(classes.index)

Label classes: ['location', 'cleanliness', 'checkin', 'accuracy', 'rating', 'value', 'communication']


In [171]:
df_train['host_2'] = df_train['host_2'].map({'t': 1, 'f':0}).astype('int')
df_train['host_3'] = df_train['host_3'].map({'t': 1, 'f':0}).astype('int')
df_train['host_4'] = df_train['host_4'].map({'t': 1, 'f':0}).astype('int')
df_train['property_15'] = df_train['property_15'].map({'t': 1, 'f':0}).astype('int')
df_train['property_10'] = df_train['property_10'].map({'t': 1, 'f':0}).astype('int')

In [172]:
df_train['host_6'] = df_train['host_6'].str.replace('%','').astype('float32')
df_train['host_7'] = df_train['host_7'].str.replace('%','').astype('float32')

In [173]:
df_train['host_6'] = df_train['host_6'] / 100
df_train['host_7'] = df_train['host_7'] / 100


In [174]:
df_train['property_price'] = df_train['property_price'].str.replace('$','')
df_train['property_price'] = df_train['property_price'].str.replace(',','').astype('float32')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [175]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85532 entries, 0 to 85531
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    85532 non-null  object 
 1   host_1                85532 non-null  int64  
 2   host_2                85532 non-null  int64  
 3   host_3                85532 non-null  int64  
 4   host_4                85532 non-null  int64  
 5   host_5                85532 non-null  object 
 6   host_6                85532 non-null  float32
 7   host_7                85532 non-null  float32
 8   host_8                85532 non-null  int64  
 9   property_1            85532 non-null  float64
 10  property_2            85532 non-null  float64
 11  property_3            85532 non-null  float64
 12  property_4            85532 non-null  object 
 13  property_5            85532 non-null  object 
 14  property_6            85532 non-null  int64  
 15  property_7         

## Training and evaluation

We will split the dataset into train and test set with a 90%-10% split percentage. I just like keeping most of the data for training purposes. You will also notice that we ditched scikit-learn and used nuumpy and pandas to do the splitting by utilizing randomness from the numpy module. Although there remains a question for reproducability since there are no random_seed param here unlike that in Scikit-learn. This something to explore in future as well

In [176]:
df_train.drop('id', axis=1, inplace=True)

In [177]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.10):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(df_train)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

77095 examples in training, 8437 examples for testing.


## Tf Datasets

#### We convert the pandas dataframes to tensorflow datasets which are more efficient and provide faster operations. Check out them [here](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

In [178]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

/opt/conda/lib/python3.7/site-packages/tensorflow_decision_forests/keras/core.py:2574: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


## Training and model selection

#### Tf-df provides a few variants learning algorithms for Decision trees namely:
- RandomForest
- GradientBoosted Trees
- CartModel,
- DistributedGradientBoostedTreesModel

##### For this notebook I tried the first two and found the second to perform better

In [179]:
# Specify the model.
# model_1 = tfdf.keras.GradientBoostedTreesModel(
#     num_trees=300,
#     growing_strategy="BEST_FIRST_GLOBAL",
#     max_depth=8,
#     split_axis="SPARSE_OBLIQUE",
#     categorical_algorithm="RANDOM",)

# # Optionally, add evaluation metrics.
# # model_1.compile(
# #     metrics=[tfa.metrics.F1Score(num_classes = len(classes))])

# # Train the model.

# model_1.fit(x=train_ds)

In [180]:
model_1 = tfdf.keras.RandomForestModel()

# Train the model.
model_1.fit(x=train_ds)

Use /tmp/tmp79lyb10q as temporary training directory
Reading training dataset...
Training dataset read in 0:00:02.282785. Found 77095 examples.
Training model...


[INFO kernel.cc:1176] Loading model from path /tmp/tmp79lyb10q/model/ with prefix 97144157319e49ab
[INFO abstract_model.cc:1248] Engine "RandomForestGeneric" built
[INFO kernel.cc:1022] Use fast generic engine


Model trained in 0:01:39.862692
Compiling model...
Model compiled.


## Evaluation
When the model finishes training you can evaluate it on the test dataset that we set aside earlier. Based on the performance here we can then decide to tune hyperparameters or change the learning algorithm to suit our needs

In [181]:
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

9/9 [==============================] - 2s 213ms/step - loss: 0.0000e+00

loss: 0.0000


## Inspecting model structures and hidden score
### The different algorithms of Decision Trees will score the features differently and assign importance, we can inspect them like so

In [182]:
# model_1.summary()

In [183]:
# model_1.make_inspector().features()

In [184]:
# model_1.make_inspector().variable_importances()

## Inspecting and Plotting the training logs 📈📉
### The training logs which are availbale through the `make_inspector()` method contain wealth of information of how the training progresses. It is more useful to plot the result and inpect the trend

In [185]:
#

In [186]:
# import matplotlib.pyplot as plt

# logs = model_1.make_inspector().training_logs()

# plt.figure(figsize=(12, 4))

# plt.subplot(1, 2, 1)
# plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
# plt.xlabel("Number of trees")
# plt.ylabel("Accuracy (out-of-bag)")

# plt.subplot(1, 2, 2)
# plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
# plt.xlabel("Number of trees")
# plt.ylabel("Logloss (out-of-bag)")

# plt.show()

### We can see that training process fluctuates quite a lot but the model converges after 25 or so number of trees

## Submission Time

In [187]:
test = pd.read_csv('/kaggle/input/was-your-stay-worth-its-price/test.csv')
id = test['id']
test.drop('id', axis=1, inplace=True)
test['host_2'] = test['host_2'].map({'t': 1, 'f':0}).astype('int')
test['host_3'] = test['host_3'].map({'t': 1, 'f':0}).astype('int')
test['host_4'] = test['host_4'].map({'t': 1, 'f':0}).astype('int')
test['property_15'] = test['property_15'].map({'t': 1, 'f':0}).astype('int')
test['property_10'] = test['property_10'].map({'t': 1, 'f':0}).astype('int')
test['host_6'] = test['host_6'].str.replace('%','').astype('float32')
test['host_7'] = test['host_7'].str.replace('%','').astype('float32')
test['property_price'] = test['property_price'].str.replace('$','')
test['property_price'] = test['property_price'].str.replace(',','').astype('float32')

test = tfdf.keras.pd_dataframe_to_tf_dataset(test)
preds = model_1.predict(test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()


47/47 [==============================] - 10s 218ms/step


In [188]:
preds

array([[0.11999995, 0.18333323, 0.23999985, ..., 0.03333334, 0.1033333 ,
        0.25333318],
       [0.14999993, 0.06000001, 0.23333319, ..., 0.00666667, 0.00333333,
        0.52333295],
       [0.503333  , 0.09999997, 0.1466666 , ..., 0.04333334, 0.02      ,
        0.13333328],
       ...,
       [0.453333  , 0.02      , 0.21999986, ..., 0.00333333, 0.00666667,
        0.28333315],
       [0.3866664 , 0.06000001, 0.18666656, ..., 0.05666667, 0.02666667,
        0.21999986],
       [0.05      , 0.03666667, 0.4666663 , ..., 0.00333333, 0.        ,
        0.41666636]], dtype=float32)

In [189]:
maxInRows = np.argmax(preds, axis=1)
maxInRows

array([6, 6, 0, ..., 0, 0, 2])

In [190]:
submission = pd.DataFrame({'id': id, 'max_rating_class': maxInRows})
submission['max_rating_class'] = submission['max_rating_class'].map({0:'location', 1:'cleanliness', 2:'checkin', 3:'accuracy', 4:'rating', 5:'value', 6:'communication'})

Finally we shouldn't forget that the model is returning a probability between 0 and 1 and we need to define a threshold above which the prediction is true and false otherwise

In [191]:
submission.to_csv('submission.csv', header=True, index=False)

## References:
* [Official Tensorflow Decision Forest Blog](https://blog.tensorflow.org/2021/05/introducing-tensorflow-decision-forests.html)
* [Official Tensorflow Page for Decision Forest](https://www.tensorflow.org/decision_forests)
